In [1]:
import json
import argparse

from utils import invoke, evaluate_time_predictions
from threadpool import ThreadPool
from tqdm import tqdm

prompt_file = "prompts/v0.txt"
bench_file = "time_bench.json"

with open(prompt_file) as f:
    prompt = f.read()

with open(bench_file) as f:
    time_bench = json.load(f)

# Function to process a single example
def process_example(example, prompt):
    params = {
        "question": example["question"],
        "current_time_iso": example["now"],
        "date_field": "timestamp",
        "other_date_fields": []
    }
    
    return invoke(prompt, **params)


# Create a thread pool
thread_pool = ThreadPool(max_workers=2)

# Use the thread pool to process all examples in parallel
predicts = thread_pool.map_with_args(
    func=process_example,
    items=time_bench,
    fixed_args={"prompt": prompt},
    desc="Processing examples",
)

print(predicts)

Processing examples: 100%|██████████| 28/28 [00:27<00:00,  1.03it/s]

[{'analysis': 'The user query is asking for the average response time for each API endpoint in the application over the last hour. This indicates a time constraint related to the \'timestamp\' time field.\n\nThe key time-related expression in the query is "over the last hour". This is a relative time constraint that needs to be converted to an absolute time range based on the current time of 2015-08-31 09:47:56.\n\nTo extract the relevant time range, we need to calculate the start and end times for the "last hour" period:\n- Start time: 2015-08-31 08:47:56 (1 hour before the current time)\n- End time: 2015-08-31 09:47:56 (the current time)\n\nThis time range is directly related to the \'timestamp\' time field and should be used as an additional filter for the query execution.', 'start': '2015-08-31 08:47:56', 'end': '2015-08-31 09:47:56'}, {'analysis': '1. Analyzing the query, we can identify the following time-related expression: "over the last 24 hours".\n\n2. This time constraint is

In [ ]:
for i in range(len(predicts)):
    for key in ["start", "end"]:
        if predicts[i][key]!=None:
            predicts[i][key] = predicts[i][key].replace("T", " ")
    
    if predicts[i]["start"] != time_bench[i]["start"]:
        print(i)
        print(time_bench[i])
        print(predicts[i])
        break

3
{'question': 'What is the distribution of different HTTP status codes in our web server logs over the last hour?', 'query': 'source=web_server_logs | where timestamp > now() - 1h | stats count() as request_count by status_code', 'now': '2015-06-14 06:06:16', 'start': '2015-06-14 05:06:16', 'end': '2015-06-14 06:06:16'}
{'analysis': 'The user query mentions "over the last hour", which is a time-related expression that applies to the specified time field "timestamp". This is a query-level time constraint that defines the search scope.\n\nThe analysis is as follows:\n1. The query contains the time-related expression "over the last hour", which refers to the time field "timestamp".\n2. This is a query-level time constraint that should be used to filter the search results, not an aggregation time reference.\n3. The relative time expression "last hour" needs to be converted to an absolute ISO time range based on the current time of 2015-06-14 06:06:16.', 'start': '2015-06-14T05:06:16', 'en

In [12]:
time_bench[i]["start"]

'2015-06-14 05:06:16'

In [10]:
predicts[i]["start"]

'None'

In [8]:
predicts[i]["start"] != time_bench[i]["start"]

True

In [14]:
print(predicts[i]["analysis"])

The user query mentions "over the last hour", which is a time-related expression that applies to the specified time field "timestamp". This is a query-level time constraint that defines the search scope.

The analysis is as follows:
1. The query contains the time-related expression "over the last hour", which refers to the time field "timestamp".
2. This is a query-level time constraint that should be used to filter the search results, not an aggregation time reference.
3. The relative time expression "last hour" needs to be converted to an absolute ISO time range based on the current time of 2015-06-14 06:06:16.
